# Load Libraries

In [1]:
import numpy          as np
import pandas         as pd
import plotly.express as px

from geopy.geocoders import Nominatim

pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Functions

In [19]:
def show_dtypes(data):
    print(data.dtypes)
    return None

def show_dimensions(data):
    print("Number of rows: {}".format(data.shape[0]), end = '\n\n')
    print("Number of columns: {}".format(data.shape[1]), end = '\n\n')
    return None

def collect_geodata(data, cols):
    # Initialize API
    geolocator = Nominatim (user_agent = 'geopiExercises')

    
    # create empty
    data.loc[:, cols[0]] = 'NA'
    data.loc[:, cols[1]] = 'NA'

    for i in range(len(data)):
        print('Loop: {}/{}'.format(i, len(data)))

        query = str(data.loc[i, 'lat']) + ',' + str(data.loc[i, 'long'])
        response = geolocator.reverse(query)

        if cols[0] in response.raw['address']:
            data.loc[i, 'house_number'] = response.raw['address'][cols[0]]

        if cols[1] in response.raw['address']:
            data.loc[i, 'road'] = response.raw['address'][cols[1]]
    
    return data

# Extract Data

In [3]:
data = pd.read_csv('datasets/kc_house_data.csv')

## Extraction Analysis

In [4]:
# data dimension
show_dimensions(data)

Number of rows: 21613
Number of columns: 21


In [5]:
show_dtypes(data)

id                 int64
date              object
price            float64
bedrooms           int64
bathrooms        float64
sqft_living        int64
sqft_lot           int64
floors           float64
waterfront         int64
view               int64
condition          int64
grade              int64
sqft_above         int64
sqft_basement      int64
yr_built           int64
yr_renovated       int64
zipcode            int64
lat              float64
long             float64
sqft_living15      int64
sqft_lot15         int64
dtype: object


# Transformation

In [6]:
data['date'] = pd.to_datetime(data['date']) 

In [7]:
# descriptive statistics
num_attributes = data.select_dtypes(include = ['int64', 'float64'])


# central tendency - media, median
media = pd.DataFrame(num_attributes.apply(np.mean, axis = 0))
mediana = pd.DataFrame(num_attributes.apply(np.median, axis = 0))

# dispersion - std, min, max
std = pd.DataFrame(num_attributes.apply(np.std, axis = 0))
min_ = pd.DataFrame(num_attributes.apply(np.min, axis = 0))
max_ = pd.DataFrame(num_attributes.apply(np.max, axis = 0))

df1 = pd.concat([max_, min_, media, mediana, std], axis = 1).reset_index()
df1.columns = ['attributes', 'maximo', 'minimo', 'media', 'mediana', 'std']

In [8]:
show_dimensions(df1)

Number of rows: 20
Number of columns: 6


In [9]:
# Calassificando os quartos

data['dormitory_type'] = 'NA'

for i in range(len(data)):
    if data.loc[i, 'bedrooms'] == 1:
        data.loc[i,'dormitory_type'] = 'studio'
    elif data.loc[i, 'bedrooms'] == 2:
        data.loc[i,'dormitory_type'] = 'apartment'
    else:
        data.loc[i, 'dormitory_type'] = 'house'


In [20]:
# data dimension
show_dimensions(data)

Number of rows: 21613

Number of columns: 23



In [21]:
cols = ['road', 'house_number']
df1 = data.head(20)
df1 = collect_geodata(df1, cols)
show_dimensions(df1)

/home/fernandoivanmeyer/.pyenv/versions/3.9.2/envs/ZeroAoDS/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/home/fernandoivanmeyer/.pyenv/versions/3.9.2/envs/ZeroAoDS/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



Loop: 0/20
Loop: 1/20
Loop: 2/20
Loop: 3/20
Loop: 4/20
Loop: 5/20
Loop: 6/20
Loop: 7/20
Loop: 8/20
Loop: 9/20
Loop: 10/20
Loop: 11/20
Loop: 12/20
Loop: 13/20
Loop: 14/20
Loop: 15/20
Loop: 16/20
Loop: 17/20
Loop: 18/20
Loop: 19/20
Number of rows: 20

Number of columns: 25



In [11]:
# Classificando os imóveis por valores

data['level'] = 'NA'

for i in range(len(data)):
    if data.loc[i, 'price'] <= 321950:
        data.loc[i, 'level'] = 0
    
    elif (data.loc[i, 'price'] > 321950) & (data.loc[i, 'price'] <= 450000):
        data.loc[i, 'level'] = 1
        
    elif (data.loc[i, 'price'] > 450000) & (data.loc[i, 'price'] <= 645000):
        data.loc[i, 'level'] = 2
    
    else:
        data.loc[i, 'level'] = 3

# Load

In [23]:
# mapa 
houses = df1[['id', 'lat', 'long', 'price', 'level']].copy()

fig = px.scatter_mapbox(houses,
                 lat = 'lat',
                 lon = 'long',
                 color = 'level',
                 size = 'price',
                 color_continuous_scale = px.colors.cyclical.IceFire,
                 size_max = 15,
                 zoom = 10)

fig.update_layout(mapbox_style = 'open-street-map')
fig.update_layout(height = 600, margin = {'r': 0, 'l': 0, 'b': 0, 't': 0} )